In [ ]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;

!apt install cuda-8.0;


39 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2021-11-24 07:15:01--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2021-11-24 07:15:01--  https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?YGHIHMr

In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zm4q62fc
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zm4q62fc
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=990bebef8be235121613f718e9ee631411aa9bc9398becbe178594e7ba50a8a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-70jrzgn1/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;

Extracting templates from packages: 100%
Selecting previously unselected package gcc-5-base:amd64.
(Reading database ... 164314 files and directories currently installed.)
Preparing to unpack .../00-gcc-5-base_5.5.0-12ubuntu1_amd64.deb ...
Unpacking gcc-5-base:amd64 (5.5.0-12ubuntu1) ...
Selecting previously unselected package libisl15:amd64.
Preparing to unpack .../01-libisl15_0.18-4_amd64.deb ...
Unpacking libisl15:amd64 (0.18-4) ...
Selecting previously unselected package cpp-5.
Preparing to unpack .../02-cpp-5_5.5.0-12ubuntu1_amd64.deb ...
Unpacking cpp-5 (5.5.0-12ubuntu1) ...
Selecting previously unselected package cuda-toolkit-config-common.
Preparing to unpack .../03-cuda-toolkit-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-config-common (11.5.117-1) ...
Selecting previously unselected package cuda-toolkit-11-config-common.
Preparing to unpack .../04-cuda-toolkit-11-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-11-config-common (11.5.117-1) ...
Selec

In [ ]:
!/usr/local/cuda/bin/gcc --version

gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
%%cu
#include <stdio.h>
__global__ void Hellokernel()
{

}
main()
{
Hellokernel << <1, 1 >> > ();
 printf("Hello World\n");
return 0;
}

Hello World



In [ ]:
%%cu
#include <stdio.h>
__global__ void add(int a, int b, int *c)
{
*c = a + b;
}
int main(void)
{
int c;
int *dev_c;
cudaMalloc((void**)&dev_c, sizeof(int));
add << <1, 1 >> > (2, 7, dev_c);
cudaMemcpy(&c, dev_c, sizeof(int),
cudaMemcpyDeviceToHost);
printf("2 + 7 = %d\n", c);
cudaFree(dev_c);
return 0;
}

2 + 7 = 1



In [ ]:
%%cu
#include <stdio.h>
__global__ void vector_add(int *out_d, int *a, int *b, int n) 
	{ 
	int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x;
int ty = threadIdx.y;
int row = by*blockDim.y + ty;
int col = bx*blockDim.x + tx;
int dim = gridDim.x*blockDim.x;
int i = row*dim + col;
out_d[i] = a[i] + b[i]; 

	 }

int main()
	{ 
	int *a, *b, *out_d,*out;
	 int *d_a, *d_b;
	 int N=64;
	int i;
	 a = (int*)malloc(sizeof(int) * N);
	 b = (int*)malloc(sizeof(int) * N);
  	 out = (int*)malloc(sizeof(int) * N);
	for (i=0;i<N;i++)
		{
		a[i]=i;
		b[i]=i*2;
		}
 	cudaMalloc((void**)&d_a, sizeof(int) * N); 
 	cudaMalloc((void**)&d_b, sizeof(int) * N); 
  cudaMalloc((void**)&out_d, sizeof(int) * N); 
	cudaMemcpy(d_a, a, sizeof(int) * N, cudaMemcpyHostToDevice); 
	cudaMemcpy(d_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
	vector_add<<<2,4>>>(out_d, d_a, d_b, N); 
  	cudaMemcpy(out, out_d, sizeof(int) * N, cudaMemcpyDeviceToHost);
	printf("Success");
  	for (i=0;i<N;i++)
		{
		printf("%d\n",out[i]);
		}
	cudaFree(d_a); 
	cudaFree(d_b); 
  	cudaFree(out_d); 
	free(a); 
	free(b); 
  	free(out); 
  return 0;
	}

Success0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0



In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
